In [1]:
#To do: Seperate models for notes (CEloss)
#To do: Seperate model for time/velocity/sustain...
#To do: Tokenized notes and then it's a classification task with a softmax at the output layer.
# Worst case do: 1 or not 1 note detector...

# Project File - APS360 Team 25
Divided into the following section: 
# 
1) Library imports
2) Data imports
3) Model architecture definition
4) Training function definition
5) Model training
6) Model testing

## Library imports 
(Place all library imports here)

In [2]:
#KP - I just added the main ones from the labs.
import torch
import numpy as np

#import torchvision
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

import time # Tracking model training time.

#for Data importing
import mido
from mido import MidiFile, Message, MidiTrack, MetaMessage
import os
import random

In [3]:
#Set working directory if required:
os.chdir('D:\engsci\year 3\CLASS\APS360\Project') #Sets current working directory!

## Data imports
#### MIDI reading functions

In [4]:
def CountTracks(directory):          #Count files and tracks in folder
    trackCount = 0
    fileCount = 0
    for file in os.listdir(directory):
        if file.endswith(".midi"):
            fileCount += 1
            midiDir = MidiFile(directory+"/"+file)
            for track in midiDir.tracks:
                trackCount += 1
    print(fileCount+" files")
    print(trackCount+" tracks")

    
def PrintMessages(mid):                # print midi messages
    for i, track in enumerate(mid.tracks):
        print('Track {}: {}'.format(i, track.name))
        for msg in track:
            print(msg)

            
def PrintSomeMessages(mid):             #print first 200 midi messages
    track = mid.tracks[1]
    for i,msg in enumerate(track):
        if i < 200:
            print(msg)
            
def PrintMetaMessages(mid):             #print fmeta messages
    track = mid.tracks[0]
    for i,msg in enumerate(track):
        print(msg)

def cleanupMessages(mid):              #removes non-note messages by force
    track = mid.tracks[1]
    track2 = []
    for msg in track:
        if msg.type == "note_on":
            track2.append(msg)
    mid.tracks[1] = track2

#### MIDI to Numpy code

In [5]:

def Midi2NumpyNoSustain(mid):                                #converts to numpy array removing non-note messages
    track = mid.tracks[1]                           #0th track only contains meta-messages, all notes on 1st track
    notes = np.empty([0,4])
    time = 0
    for msg in track:
        if msg.type == "note_on":                   # only count "note" messages - other inputs i.e. foot pedals are ignored
            notes = np.append(notes,np.array([[msg.note, msg.velocity, msg.time + time, 0]]),axis=0)         # (note, velocity, time, sustain)
            time = 0
        else:
            time += msg.time                        #adjust time when removing other messages
    return notes


def NumpyGetSustain(note):
    notes = np.copy(note)
    for i, msg in enumerate(notes):
        if msg[1] > 0:                            # if velocity is not 0
            j = 1
            sustain = 0
            while msg[0] != notes[i+j][0]:        # while note values are different
                sustain += notes[i+j][2]
                j += 1                            #search for next message with same note i.e. message telling that note was released
            notes[i,3] = sustain + notes[i+j][2]
    time = 0
    for i, msg in enumerate(notes):
        if msg[1] > 0:
            notes[i,2] += time
            time = 0
        else:
            time += msg[2]                        #adjust time
    notes = notes[notes[:,1] > 0]                 #filter for notes with positive velocities (note presses)
    return notes

def NumpyNormalize(note, oneHot=False):                         #normalize all values to 0-1
    notes = np.copy(note)
    
    if oneHot:
        notes[:,12] /= 11
        notes[:,13] /= 128
        notes[:,14] /= 40000
        notes[:,15] /= 40000
    else:
        notes[:,0] /= 128
        notes[:,1] /= 128
        notes[:,2] /= 40000
        notes[:,3] /= 40000       
    return notes

def NumpyOneHot(note):
    notes = np.copy(note)
    oneHot = np.zeros([len(notes),16])
    oneHot[:, 13:] = notes[:, 1:]
    names = notes[:,0]
    namesOct = names%12
    oneHot[:,12] = (names-(namesOct))/12
    
    for i, name in enumerate(namesOct):
        oneHot[i,name.astype(int)] = 1
    
    return oneHot

def Midi2Numpy(path, oneHot=False): # full midi to numpy conversion
    mid = MidiFile(path)
    notes = Midi2NumpyNoSustain(mid)
    cleanNotes = NumpyGetSustain(notes)
    
    if oneHot:
        cleanNotes = NumpyOneHot(cleanNotes)
    
    normNotes = NumpyNormalize(cleanNotes, oneHot=oneHot)
    return normNotes

#### Numpy to MIDI code

In [6]:

def NumpyDenormalize(note): # interpret all values from 0-1 to normal values
    notes = np.copy(note)    
    if notes.shape[1] == 16: # if encode as one-hot
        notes[:,12] *= 11
        notes[:,13] *= 128
        notes[:,14] *= 40000
        notes[:,15] *= 40000
        
        notes = NumpyEncode(notes) #encode back as original 4-variable format
    else:
        notes[:,0] *= 128
        notes[:,1] *= 128
        notes[:,2] *= 40000
        notes[:,3] *= 40000       
    return notes.astype(int)

def NumpyEncode(note): # convert back from one-hot encoding
    notes = np.copy(note)
    encoded = np.zeros([len(notes),4])
    encoded[:, 1:] = notes[:, 13:]
    encoded[:, 0] = notes[:,12]*12
    
    for i in range(len(notes)):
        encoded[i,0] += np.argmax(notes[i,:12])
    
    return encoded

def NumpySequence(notes): # put all notes into a "timeline" i.e.: time values of [10, 20, 10, 30] become [10, 30, 40, 70]
    sequenced = np.copy(notes)                      # this allows us to easily add vel=0 notes in any order since we can later sort them by time
    for i, msg in enumerate(sequenced):
        if i > 0:
            sequenced[i,2] += sequenced[i-1,2]
    return sequenced

def NumpyAddOffNotes(sequenced): # add vel=0 notes from sustain into sequenced timeline
    withOff = np.copy(sequenced)
    for msg in sequenced:
        offNote = np.array([[msg[0], 0, msg[2] + msg[3], 0]])
        withOff = np.append(withOff, offNote, axis=0)
    #withOff = np.sort(withOff,axis=0)
    withOff = withOff[withOff[:,2].argsort()] # sort by time
    return withOff

def NumpyUnsequence(notes): # revert time value to "time since last message"
    unsequenced = np.copy(notes)
    for i, msg in reversed(list(enumerate(unsequenced))):
        unsequenced[i,3] = 0
        if i > 0:
            unsequenced[i,2] -= unsequenced[i-1,2]
    return unsequenced

def Numpy2MidiDirect(array):    #make MIDI object from numpy
    #Start with initializing a new Mido Track:
    mid = MidiFile()
    track0 = MidiTrack()
    track1 = MidiTrack()
    
    track0.append(MetaMessage('set_tempo', tempo=500000, time=0)) #MetaMessages not necessary but are present in used files
    track0.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))
    track0.append(MetaMessage('end_of_track', time=1))
    
    track1.append(Message('program_change', channel=0, program=0, time=0))
    
    for i,note in enumerate(array):         # Get the index and the note. Array must be int array
        j = 1
        track1.append(Message('note_on',note = array[i,0], velocity = array[i,1],time = array[i,2])) # Add the note to the track.

    mid.tracks.append(track0)
    mid.tracks.append(track1)
    return mid

def Numpy2Midi(notes, name): # full numpy to midi conversion, saving result to [name].midi
    denorm = NumpyDenormalize(notes)
    seq = NumpySequence(denorm)
    off = NumpyAddOffNotes(seq)
    unseq = NumpyUnsequence(off)
    mid = Numpy2MidiDirect(unseq)
    mid.save(name + ".midi")

#### Generating tensor dataset from CSVs

In [7]:
def Numpy2Dataset(notes,num=100,skip=10): # make list of sumpy arrays #Playing with window sized (num)
    samples = []
    i = 0
    while i+num <= len(notes):
        samples.append(notes[i:i+num])
        i += skip
    return samples

def SampleAllNumpy(dataPath): # generate samples from all saved CSVs
    allSamples = []

    for i,f in enumerate(os.listdir(dataPath)):
        
        
        if i % 1280 == 0:
            print(i)
            notes = np.genfromtxt(dataPath+f, delimiter=',') #Moved down... 
            allSamples += Numpy2Dataset(notes) #Moved down into if statement...
    
    return allSamples

def SaveSamplesTensor(samples, outputPath): # save tensor
    tens = torch.Tensor(samples)
    torch.save(samples, outputPath+"Notes_Dataset_test_onehot.pt")
    return tens   

def SaveAllSamples(dataPath, outputPath): # save dataset tensor
    samples = SampleAllNumpy(dataPath)
    SaveSamplesTensor(samples, outputPath)
    print('done')

#### Bulk data conversion code - COMMENT OUT IF NOT IN USE!!!

In [8]:
# SaveAllSamples("D:/engsci/year 3/CLASS/APS360/Project/data/numpy_files/","D:/engsci/year 3/CLASS/APS360/Project/data/") #save all into tensor

## Baseline Model Code
#### getting available notes

In [9]:
def GetAllNotesMajor(root):# Get all used notes in major scale of root=root
    notes = []
    intervals = [2,2,1,2,2,2,1]
    
    while root > 24: #bring down to lowest used octave
        root -= 12
    
    n = root
    notes.append(n)
    while n < 84: #up to higherst used note
        for i in intervals:
            n += i
            notes.append(n)   
    return notes    


def GetRangeMajor(notes, low, high): # Get all notes within range
    lowIndex = notes.index(low)
    highIndex = notes.index(high)
    
    return notes[lowIndex:highIndex+1]   

#### Piece Class
##### represents whole output from all 4 voices

In [10]:
class Piece: # Entire baseline model compostion - composed of 4 voices soprano, alto, tenor, bass (SATB)
    def __init__(self, barNum=16, root=60):# 16 bars in C major
        self.root = root # root note
        self.allNotes = GetAllNotesMajor(self.root) # all notes on major scale
        self.barNum = barNum # number of bars
        
        self.soprano = Voice(self.allNotes,60,84,speed=8) # SATB
        self.alto = Voice(self.allNotes,48,72)
        self.tenor = Voice(self.allNotes,36,60)
        self.bass = Voice(self.allNotes,24,48)
          
        self.notes = np.empty([0,4]) #notes output
        
        self.pieceChords = [] # chords
        
        self.chords = np.array([ # common classical C major chords
            [ 0,  4,  7,  0],# I
            [ 2,  5,  9,  2],# ii
            [ 4,  7, 11,  4],# iii
            [ 5,  9, 0,  5],# IV
            [ 7, 11, 2,  7],# V
            [ 9, 0, 4,  9],# vi
            [11, 2, 5, 11],# vii dim
            [ 2,  5,  9, 0],# ii7
            [ 5,  9, 0, 4],# IVmaj7
            [ 7, 11, 2, 5],# V7
            [11, 2, 5, 9]])# vii7 half-dim
        
    def GenerateSoprano(self): # Generate soprano line
        self.soprano.GenerateLine(self.soprano.speed*self.barNum)
        
    def GenerateAlto(self): # Generate alto line from chords
        self.alto.GenerateChordLine(self.pieceChords)
        
    def GenerateTenor(self): # see alto
        self.tenor.GenerateChordLine(self.pieceChords)
        
    def GenerateBass(self): # see alto
        self.bass.GenerateChordLine(self.pieceChords)
        
        
    
    def ChooseChord(self, sopNote): # Choose a fitting chord for soprano note
        while sopNote >= 12:
            sopNote -= 12
        
        goodChords = np.empty([0,4])
        
        for chord in self.chords:
            if (chord==sopNote).sum() > 0:
                goodChords = np.append(goodChords,[chord],axis=0)
        
        chosenChord = goodChords[random.randint(0,len(goodChords)-1)]
        chosenChord = np.sort(np.unique(chosenChord))
        
        i = 12
        chordNotes = chosenChord
        while i < 120:
            chordNotes = np.append(chordNotes, chosenChord+i)
            i += 12
        
        return(chordNotes)
    
    def GetChords(self): # select all chords in piece
        for i, note in enumerate(self.soprano.notes):
            if i % 2 == 0:
                sopNote = note[0]
                chord = self.ChooseChord(sopNote)
                self.pieceChords.append(chord)
                
    def GenerateLines(self): # Generate all SATB lines and joins them - entire baseline model
        self.GenerateSoprano()
        self.GetChords()
        self.GenerateAlto()
        self.GenerateTenor()
        self.GenerateBass()
        self.joinLines()
        
        self.notes = self.notes.astype(int)
        self.OffsetTime(20)
        
        return self.notes
        
    def InsertLine(self, starting, inserted, startIndex, skipIndex): # join 2 lines
        base = np.copy(starting)
        ins = np.copy(inserted)
        
        for i,note in enumerate(ins):
            base = np.insert(base, (i*skipIndex)+startIndex, [note], axis=0)
            
        return base
        
    def joinLines(self): # join all SATB lines
        #self.notes = np.copy(self.soprano)
        self.notes = self.InsertLine(self.soprano.notes, self.alto.notes, 1, 3)
        self.notes = self.InsertLine(self.notes, self.tenor.notes, 2, 4)
        self.notes = self.InsertLine(self.notes, self.bass.notes, 3, 5)
        
    def OffsetTime(self, maxChange): # adds random time offsets to make output more organic
        for note in self.notes:
            note[2] += random.randint(0,maxChange)
        

#### Voice class
##### Represents individual voices

In [11]:
class Voice: # individual voices
    def __init__(self, allNotes, lowNote, highNote, jump=3, speed=4, time=4096, velocity=64):
        self.range = GetRangeMajor(allNotes,lowNote,highNote) #available ntoes
        self.jump = jump #maximum pitch interval between notes
        self.speed = speed #note length i.e. 4 for quarter, 8 for eighth etc.
        self.time = time #song speed
        self.velocity = velocity #note volume
        self.notes = np.empty([0,4]) #notes output
        self.lowNote = lowNote # lowest note
        self.highNote = highNote # highest note
        self.allNotes = allNotes # all notes in scale
            
        self.duration = self.time / self.speed # time between notes
        
        
    def RandomStartNote(self): # Generate Random first note (for soprano)
        note = random.choice(self.range)
        self.notes = np.append(self.notes,np.array([[note, self.velocity, 0, self.duration]]),axis=0)
        
        
    def RandomJump(self): # Generate Random next note (for soprano)
        lastNote = self.notes[len(self.notes)-1][0] # find last played note
        lastIndex = self.range.index(lastNote)
        
        newIndex = -1
        while newIndex < 0 or newIndex >= len(self.range): # stay in range
            newIndex = lastIndex + random.randint(-self.jump,self.jump)
            
        newNote = self.range[newIndex]
        self.notes = np.append(self.notes,np.array([[newNote, self.velocity, self.duration, self.duration]]),axis=0)
        
        
    def GenerateLine(self, length): # Generate random line (for soprano)
        self.RandomStartNote()
        
        for n in range(length-1):
            self.RandomJump()
            
            
    def clearNotes(self):
        self.notes = np.empty([0,4])
        
    def GetChordNotes(self, chordNotes): # Get useful notes from all chord notes
        chordNotes = chordNotes[chordNotes >= self.lowNote]
        chordNotes = chordNotes[chordNotes <= self.highNote]
        return chordNotes
    
    def ChooseStartChordNote(self, chordNotes): # Choose Random note in chord
        note = random.choice(chordNotes)
        self.notes = np.append(self.notes,np.array([[note, self.velocity, 0, self.duration]]),axis=0)
        
    def ChooseChordNote(self,chordNotes): # Choose suitable next note in chord
        lastNote = self.notes[len(self.notes)-1][0] # find last played note
        
        chordNotes = chordNotes[chordNotes >= lastNote - (self.jump*2)]
        chordNotes = chordNotes[chordNotes <= lastNote + (self.jump*2)]
        newNote = random.choice(chordNotes)
        
        self.notes = np.append(self.notes,np.array([[newNote, self.velocity, 0, self.duration]]),axis=0)
        
    def GenerateChordLine(self, chords): # Generate A/T/B lines
        
        firstChord = self.GetChordNotes(chords[0])
        self.ChooseStartChordNote(firstChord)
        
        for c in chords[1:]:
            chord = self.GetChordNotes(c)
            self.ChooseChordNote(chord)

## Model architecture definition

# Regression based model

In [12]:

torch.manual_seed(1)
class MusicGenerator_classifier(nn.Module):
    def __init__(self, input_size=12, hidden_size=32,num_layers = 12,num_classes = 2):
        super(MusicGenerator_classifier, self).__init__()
        self.name = "SergModel"
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.rnn = nn.LSTM(self.input_size, self.hidden_size, batch_first = True,num_layers = num_layers,bidirectional = False)
        #^^Expects data as: (batch_size,sequence,features)
        self.fc1 = nn.Linear(self.hidden_size, 32)
        self.fc2 = nn.Linear(32,self.num_classes)
        self.sig = nn.Sigmoid()
        #self.linear = nn.Linear(self.hidden_size, 4) #Regression problem.

    def forward(self, x):
        h0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size)
        h0 = h0.cuda()
        c0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size)
        c0 = c0.cuda()
        out,(h0,c0) = self.rnn(x,(h0,c0))
#         x = F.relu(out)
        x = F.relu(self.fc1(torch.max(out, dim=1)[0])) # 2 fully connected layers #TODO: torch.max(out,dim=1)[0] instead...
        x = self.fc2(x)
#         x = self.sig(x)
        pred = x
        #pred[:,14] = pred[:,14]**6
        
        return pred
    
print('Model class created succesfully')

Model class created succesfully


## Training function

### trainDataClass has: 9840 samples, ~17 classes?
4 feature inputs

In [13]:
trainData = list(torch.load(r'D:\engsci\year 3\CLASS\APS360\project_pivot\dataset\trainDataClass.pt'))
print(len(trainData))
print(trainData[1][1])
valData = list(torch.load(r'D:\engsci\year 3\CLASS\APS360\project_pivot\dataset\trainDataClass.pt'))
#testData = list(torch.load("data/testData.pt"))
batch_size = 2048
learning_rate = 1e-3


9840
tensor(1.)


In [14]:
def GetAccuracy_classifier(model, train=False, limit=-1):
    if train:
        data = trainData
    else:
        data = valData
        
    #err = 0
    total = 0
    correct = 0
    for imgs, labels in torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True):
#         print('imgs.shape accuracy: ',imgs.shape)
        imgs = imgs.cuda()
        ''' 
        Remove below if statement once binary classification succesful :)
        '''
#         print(labels)
        for i, label in enumerate(labels):
            if label > 0:
#                 print(label)
                labels[i]=1
#             print(labels)
        labels = labels.cuda()
          
        output = model(imgs) #Outputs a label classification...
#         print(output)
        pred = output.max(1, keepdim=True)[1] #Collects label classification
        
        pred = pred.float()
        print(pred)
        #select index with maximum prediction score
#         lab = labels.view_as(output)
        
        correct += pred.eq(labels).sum().item()
        
        #error = abs(output-lab).sum().item()
        
        #err += error/16
        total += imgs.shape[0]
        #print(total/12)
        
        if limit > 0 and (total) > limit:
            break
        
    #totalError = err / total
    accuracy = correct/total
#     accuracy -= 5/6
#     accuracy *= 6
    return accuracy

In [15]:
#To help us save the model easier...
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path


In [16]:
def train_classifier(model, batch_size=batch_size, learning_rate=learning_rate, num_epochs=1, model_name="model"):
    #https://stackoverflow.com/questions/55549843/pytorch-doesnt-support-one-hot-vector --> For one/multi hot encode to class labels
    trainLoader = torch.utils.data.DataLoader(trainData, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss() 
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # Cross-Entropy Loss and SGD are commonly used for classification problems
    
    dataSize = len(trainData)*num_epochs

    iters, losses, train_acc, val_acc = [], [], [], []
    
    maxVal = 0
    
    if torch.cuda.is_available():
        print("using cuda")

    # training
    n = 0 # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(trainLoader):
            imgs = imgs
#             print(imgs.shape)
            ''' 
            Remove below if statement once binary classification succesful :)
            '''
#             print(labels)
            for i, label in enumerate(labels):
                if label > 0:
#                     print(label)
                    labels[i]=1
#                 print(labels)
            labels = labels.long().clone().detach()
#             print(labels.shape)
#             print(labels.shape)
#             print('labels: ',labels)
            if torch.cuda.is_available():
                imgs = imgs.cuda()
                labels = labels.cuda()
            
            out = model(imgs)             # forward pass
#             print(out.shape)
#             print(out)
            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch

            # save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            
            trainAccuracy = GetAccuracy_classifier(model, train=True, limit=10000)
            train_acc.append(trainAccuracy) # compute training accuracy
            print('trainAccuracy: ',trainAccuracy)
            valAccuracy = GetAccuracy_classifier(model, train=False)
            val_acc.append(valAccuracy)  # compute validation accuracy
            
            if valAccuracy > maxVal and valAccuracy > 0.2: #Only save model when it improves.
                maxVal = valAccuracy
                torch.save(model.state_dict(), model_name+"_classifier.pt")
                print("Validation record in iteration {}: {}".format(n, maxVal)) # save model when validation accuracy is higher than previous runs
                
            if trainAccuracy > valAccuracy + 0.2: #check for overfitting
                break
            
            
            n += 1
#             if n % 10 == 0:
#                 perc = n*batch_size/dataSize
#                 print("{:.2%} complete".format(perc))
        print('epoch: {}, loss: {}'.format(epoch,losses[n-1]))

    # plotting
    plt.title("Training Curve")
    plt.plot(iters, losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Training Curve")
    plt.plot(iters, train_acc, label="Train")
    plt.plot(iters, val_acc, label="Validation")
    plt.xlabel("Iterations")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
    
    return True

## Model Training

# Classifier model

In [ ]:
## Change the learning rate to be smaller...
torch.manual_seed(1)
batch_size = 100
learning_rate = 1e-2
print(len(trainData))
start = time.time()
model_classifier = MusicGenerator_classifier(input_size = 4, hidden_size =20,num_layers = 1,num_classes = 2) #num_layers should match num_classes apparently...
model_classifier = model_classifier.cuda()
train_classifier(model_classifier,num_epochs=100)
# end = time.time()
# print(end-start)
model_classifier = model_classifier.cpu()

9840
using cuda
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
    

In [ ]:
# model = MusicGenerator(hidden_size = 64,num_layers = 3).cuda()
# params = torch.load("D:\engsci\year 3\CLASS\APS360\Project\model_hiddensize64_numlayers3_bidirectional.pt")
# model.load_state_dict(params)


In [ ]:
# print(model)
# pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# print(pytorch_total_params)

In [ ]:
#To do: Implement model loading...

## Model Testing

Since our model is 'tested' with people listening to it, we need to just generate some samples.
To do so, and because this is a regression style problem, sampling will not be needed. Instead, we will feed it a random set of starting notes from a sample of music.

TO DO: Modify for 100 notes.